# Demonstation of raytracing in Kotlin

In [51]:
%use lets-plot

In [60]:
class Point(
    var x: Float = 0f,
    var y: Float = 0f,
    var z: Float = 0f
) {
    fun getPointOnScreen(cameraPoint: Point): Pair<Float, Float> {

        if (z < 0) {
            throw IllegalArgumentException("The point is behind the screen")
        }

        // The focal length is the distance between the camera and the screen.
        val focalLength = cameraPoint.z * -1

        // The distance between the point and the camera.
        val zDistance = z - cameraPoint.z

        // the x and y coordinates of the point on the screen.
        val xProjected = ((x - cameraPoint.x) * focalLength) / (focalLength + zDistance)

        val yProjected = ((y - cameraPoint.y) * focalLength) / (focalLength + zDistance)

        return Pair(xProjected, yProjected)
    }
}

class Line {
    val points: List<Point>

    constructor(startPoint: Point, endPoint: Point) {
        points = listOf(startPoint, endPoint)
    }

    fun getPointsOnScreen(cameraPoint: Point): List<Pair<Float, Float>> {
        return points.map { it.getPointOnScreen(cameraPoint) }
    }

    fun getGeomPath(cameraPoint: Point): geomPath {
        val pointsOnScreen = getPointsOnScreen(cameraPoint)
        return geomPath(
            mapOf(
                "x" to pointsOnScreen.map { it.first },
                "y" to pointsOnScreen.map { it.second })
        )
    }
}

class Square {
    val points: List<Point>

    constructor(centerPoint: Point, size: Float) {
        points = listOf(
            Point(centerPoint.x + size, centerPoint.y - size, centerPoint.z),
            Point(centerPoint.x + size, centerPoint.y + size, centerPoint.z),
            Point(centerPoint.x - size, centerPoint.y + size, centerPoint.z),
            Point(centerPoint.x - size, centerPoint.y - size, centerPoint.z),
            Point(centerPoint.x + size, centerPoint.y - size, centerPoint.z + size),
            Point(centerPoint.x + size, centerPoint.y + size, centerPoint.z + size),
            Point(centerPoint.x - size, centerPoint.y + size, centerPoint.z + size),
            Point(centerPoint.x - size, centerPoint.y - size, centerPoint.z + size),
        )
    }

    fun getPointsOnScreen(cameraPoint: Point): List<Pair<Float, Float>> {
        return points.map { it.getPointOnScreen(cameraPoint) }
    }
}

// Defines the point of the camera in 3D space.
// The camera has to be in front if the screen whitch means that the z coordinate has to be negative.
val camera = Point(2f, 0f, -1f)


// Create two lines in the 3D space
val line = Line(Point(0f, 5f, 0f), Point(0f, 0f, 0f))
val line2 = Line(Point(0f, 5f, 4f), Point(0f, 0f, 4f))

// Create a 2d plot that represents the screen with x and y axis
val plot = letsPlot() { x = "x"; y = "y" }

// Plot the lines on the screen
plot + line.getGeomPath(camera) + line2.getGeomPath(camera)


